SyntaxError: invalid syntax (1556775364.py, line 2)

In [3]:
# %% [markdown]
# # Analyse Exploratoire des Données Spatio-Temporelles
# ## Solution de Sécurité Personnelle - Module IA
# > Dataset : NYPD Complaint Data Historic

# %% [code]
# Importations essentielles
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, MarkerCluster
import seaborn as sns
import osmnx as ox
from sklearn.preprocessing import LabelEncoder


# Configuration
sns.set(style="whitegrid", palette="pastel")
plt.rcParams['figure.figsize'] = (12, 8)


ModuleNotFoundError: No module named 'geopandas'

In [4]:

# %% [code]
# Chargement des données
print("⏳ Téléchargement des données...")
url = "https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD"
crime_data = pd.read_csv(url, low_memory=False)

⏳ Téléchargement des données...


IncompleteRead: IncompleteRead(1041711145 bytes read)

In [ ]:

# Filtrage des colonnes pertinentes
cols = [
    'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'OFNS_DESC', 
    'Latitude', 'Longitude', 'BORO_NM'
]
crime_data = crime_data[cols].dropna()

print(f"✅ Données chargées : {crime_data.shape[0]} incidents")

In [ ]:

# %% [code]
# Prétraitement temporel
crime_data['datetime'] = pd.to_datetime(
    crime_data['CMPLNT_FR_DT'] + ' ' + crime_data['CMPLNT_FR_TM'],
    errors='coerce'
)

# Extraction des composantes temporelles
crime_data = crime_data.dropna(subset=['datetime'])
crime_data['heure'] = crime_data['datetime'].dt.hour
crime_data['jour_semaine'] = crime_data['datetime'].dt.dayofweek  # 0=lundi, 6=dimanche
crime_data['mois'] = crime_data['datetime'].dt.month

print("🕒 Composantes temporelles extraites")


In [ ]:



# %% [code]
# Analyse temporelle
plt.figure(figsize=(14, 6))

# Distribution horaire
plt.subplot(1, 2, 1)
sns.histplot(crime_data['heure'], bins=24, kde=True, color='crimson')
plt.title('Distribution des Incidents par Heure')
plt.xlabel('Heure de la Journée')

# Distribution par jour de semaine
plt.subplot(1, 2, 2)
sns.countplot(x='jour_semaine', data=crime_data, palette='viridis')
plt.title('Incidents par Jour de Semaine')
plt.xlabel('Jour (0=Lundi, 6=Dimanche)')

plt.tight_layout()
plt.savefig('./temporal_analysis.png', dpi=300)
plt.show()

In [ ]:


# %% [code]
# Analyse des types de crimes
top_crimes = crime_data['OFNS_DESC'].value_counts().nlargest(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=top_crimes.values, y=top_crimes.index, palette='rocket')
plt.title('Top 10 des Types de Crimes')
plt.xlabel('Nombre d\'Occurrences')
plt.tight_layout()
plt.savefig('./crime_types.png', dpi=300)
plt.show()


In [ ]:

# %% [code]
# Cartographie des hotspots
print("🗺️ Génération de la carte des hotspots...")

# Création de la carte de base
nyc_coords = [40.7128, -74.0060]
m = folium.Map(location=nyc_coords, zoom_start=11, tiles='CartoDB dark_matter')

# Couche Heatmap
heat_data = crime_data[['Latitude', 'Longitude']].values.tolist()
HeatMap(heat_data, radius=15, gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(m)

# Ajout des clusters pour exploration détaillée
marker_cluster = MarkerCluster().add_to(m)
sample_data = crime_data.sample(1000)  # Échantillon pour performances

for idx, row in sample_data.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['OFNS_DESC']} - {row['datetime'].strftime('%Y-%m-%d %H:%M')}",
        icon=folium.Icon(color='orange', icon='exclamation-triangle')
    ).add_to(marker_cluster)

# Sauvegarde
m.save('./crime_hotspots.html')
print("✅ Carte sauvegardée : crime_hotspots.html")

In [ ]:


# %% [code]
# Analyse par arrondissement
plt.figure(figsize=(10, 6))
crime_data['BORO_NM'].value_counts().plot.pie(
    autopct='%1.1f%%',
    colors=sns.color_palette('Set3'),
    wedgeprops={'edgecolor': 'black', 'linewidth': 1.5}
)
plt.title('Répartition des Crimes par Arrondissement')
plt.ylabel('')
plt.savefig('./borough_distribution.png', dpi=300)
plt.show()


ModuleNotFoundError: No module named 'geopandas'

In [ ]:

# %% [code]
# Préparation pour modèles ML
print("⚙️ Préparation des données pour modèles d'anomalie...")

# Encodage des catégories
le = LabelEncoder()
crime_data['crime_encoded'] = le.fit_transform(crime_data['OFNS_DESC'])

# Création de features
features = crime_data[['heure', 'jour_semaine', 'Latitude', 'Longitude', 'crime_encoded']]

# Exemple de visualisation des corrélations
plt.figure(figsize=(10, 8))
sns.heatmap(features.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Corrélations entre Variables')
plt.savefig('./correlation_matrix.png', dpi=300)
plt.show()

print("🎉 Analyse complète terminée ! Prochaines étapes :")
print("- Entraîner un modèle IsolationForest sur ces données")
print("- Intégrer des données météo (NOAA) pour amélioration")

In [ ]:
# API NOAA pour données historiques
import requests
api_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
params = {
    'datasetid': 'GHCND',
    'stationid': 'GHCND:USW00094728',  # Station NYC
    'startdate': '2020-01-01',
    'enddate': '2020-12-31'
}
headers = {'token': 'VOTRE_CLE_API'}
weather_data = requests.get(api_url, params=params, headers=headers).json()

In [ ]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(n_estimators=200, contamination=0.05)
crime_data['anomaly_score'] = model.fit_predict(features)

In [ ]:
# DBSCAN pour clustering spatial
from sklearn.cluster import DBSCAN
spatial_clusters = DBSCAN(eps=0.01, min_samples=50).fit(features[['Latitude', 'Longitude']])